In [90]:
import tensorflow as tf

In [91]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [92]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval(feed_dict={f: 1})
    print(result)

1


In [93]:
result

array(1, dtype=int32)

In [94]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [95]:
housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_with_bias = np.c_[np.ones((m,1)), housing.data]

In [96]:
X = tf.constant(housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="X")
XT = tf.transpose(X)
theta = tf.matmul(
    tf.matmul(
        tf.matrix_inverse(
            tf.matmul(XT, X)
        ),
    XT),
y)

In [97]:
with tf.Session() as sess:
    [theta_value] = sess.run([theta])

In [98]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [99]:
housing

{'DESCR': 'California housing dataset.\n\nThe original database is available from StatLib\n\n    http://lib.stat.cmu.edu/datasets/\n\nThe data contains 20,640 observations on 9 variables.\n\nThis dataset contains the average house value as target variable\nand the following input variables (features): average income,\nhousing average age, average rooms, average bedrooms, population,\naverage occupation, latitude, and longitude in that order.\n\nReferences\n----------\n\nPace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,\nStatistics and Probability Letters, 33 (1997) 291-297.\n\n',
 'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.     

## Gradient Descent
### Manually

In [100]:
from sklearn.preprocessing import StandardScaler
scaled_housing_data_with_bias = StandardScaler().fit_transform(housing_data_with_bias)

In [101]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [102]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    print("Epoch", n_epochs, "MSE =", mse.eval())
    best_theta = theta.eval()

Epoch 0 MSE = 8.1150465
Epoch 100 MSE = 4.8860354
Epoch 200 MSE = 4.8396173
Epoch 300 MSE = 4.8292665
Epoch 400 MSE = 4.8224063
Epoch 500 MSE = 4.817423
Epoch 600 MSE = 4.8137693
Epoch 700 MSE = 4.8110876
Epoch 800 MSE = 4.8091063
Epoch 900 MSE = 4.8076496
Epoch 1000 MSE = 4.8065658


### Autodiff

In [103]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

In [104]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    print("Epoch", n_epochs, "MSE =", mse.eval())
    best_theta = theta.eval()

Epoch 0 MSE = 8.353867
Epoch 100 MSE = 5.0926166
Epoch 200 MSE = 4.93796
Epoch 300 MSE = 4.8966713
Epoch 400 MSE = 4.8706183
Epoch 500 MSE = 4.8520327
Epoch 600 MSE = 4.8386106
Epoch 700 MSE = 4.828902
Epoch 800 MSE = 4.8218737
Epoch 900 MSE = 4.816786
Epoch 1000 MSE = 4.813096


### Optimizer

In [126]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
mse = tf.reduce_mean(tf.square(error), name="mse")
training_op = optimizer.minimize(mse)
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
saver = tf.train.Saver()

In [127]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval(feed_dict={X: scaled_housing_data_with_bias, y: housing.target.reshape(-1,1)}))

        summary_str = mse_summary.eval(feed_dict={X: scaled_housing_data_with_bias, y: housing.target.reshape(-1,1)})
        step = epoch
        file_writer.add_summary(summary_str, step)
        sess.run(training_op, feed_dict={X: scaled_housing_data_with_bias, y: housing.target.reshape(-1,1)})

    best_theta = theta.eval()
    save_path = saver.save(sess, "/Users/henryaddison/my_model_final.ckpt")

Epoch 0 MSE = 8.449053
Epoch 100 MSE = 5.0091963
Epoch 200 MSE = 4.9036345
Epoch 300 MSE = 4.875345
Epoch 400 MSE = 4.8576202
Epoch 500 MSE = 4.8446245
Epoch 600 MSE = 4.8349066
Epoch 700 MSE = 4.8275905
Epoch 800 MSE = 4.822065
Epoch 900 MSE = 4.8178706


### Mini-batch

In [118]:
def fetch_batch(epoch, batch_index, batch_size):
    X_batch = housing_data_with_bias[batch_index*batch_size:batch_index*(batch_size+1), :]
    y_batch = housing.target.reshape(-1,1)[batch_index*batch_size:batch_index*(batch_size+1), :]
    return X_batch, y_batch

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
mse = tf.reduce_mean(tf.square(error), name="mse")
training_op = optimizer.minimize(mse)
training_op = optimizer.minimize(mse)
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
saver = tf.train.Saver()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)

            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    save_path = saver.save(sess, "/Users/henryaddison/my_model_final.ckpt")

In [128]:
with tf.Session() as sess:
    saver.restore(sess, "/Users/henryaddison/my_model_final.ckpt")
    print(theta.eval())

INFO:tensorflow:Restoring parameters from /Users/henryaddison/my_model_final.ckpt
[[-0.80491114]
 [ 0.87952006]
 [ 0.15557787]
 [-0.30842516]
 [ 0.31857252]
 [ 0.00824518]
 [-0.04350524]
 [-0.5845639 ]
 [-0.5585413 ]]


# Exercises
## 12

In [188]:
import os

import numpy as np
import tensorflow as tf

from sklearn.datasets import make_moons



In [147]:
# dataset size
m = 10_000

data, labels = make_moons(n_samples = m, noise=0.15, random_state=42)
labels = labels.reshape(-1,1)

# number of features for a data point
n = len(data[0])

# add bias term
data = np.c_[np.ones((m,1)), data]

data_train = data[0:8_000, :]
labels_train = labels[0:8_000, :]

data_test = data[8_000:, :]
labels_test = labels[8_000:, :]

In [192]:
def logistic_regression(X, y, learning_rate=0.001):
    n_inputs_including_bias = int(X.get_shape()[1])
    with tf.name_scope("model"):
        theta = tf.Variable(tf.random_uniform([n_inputs_including_bias, 1], -1.0, 1.0), name="theta")
        p_hat = tf.sigmoid(tf.matmul(X, theta), name="p_hat")

    with tf.name_scope("train"):
        loss = tf.losses.log_loss(y, p_hat, scope="loss")
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss)
        loss_summary = tf.summary.scalar('Loss', loss)

    with tf.name_scope("init"):
        init = tf.global_variables_initializer()

    with tf.name_scope("save"):
        saver = tf.train.Saver()

    return p_hat, loss, training_op, loss_summary, init, saver

tf.reset_default_graph()

epochs = 10000
learning_rate = 0.001


now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

checkpoint_path = "./ch09-tf-log-reg/model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
y_proba, loss, training_op, loss_summary, init, saver = logistic_regression(X, y, learning_rate=learning_rate)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
with tf.Session() as sess:

    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path, 'rb') as f:
            start_epoch = int(f.read())
        print("continuing training from epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        sess.run(init)
        start_epoch = 0
    
    for epoch in range(start_epoch, epochs):
        _, loss_val, summary_str = sess.run([training_op, loss, loss_summary], feed_dict={X: data_train, y: labels_train})
        if epoch % 100 == 0:
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
            print("Epoch", epoch, "loss:", loss_val)
            step = epoch
            file_writer.add_summary(summary_str, step)

continuing training from epoch 9901
INFO:tensorflow:Restoring parameters from ./ch09-tf-log-reg/model.ckpt
